##Setting

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score
from sklearn.neighbors import NearestNeighbors
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sentence_transformers import SentenceTransformer
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from imblearn.over_sampling import SMOTE
import warnings

warnings.filterwarnings('ignore')

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    data_dir = '/content/drive/MyDrive/data/'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print(f"Google Drive에 '{data_dir}' 디렉토리를 생성했습니다.")
except ImportError:
    print("Google Colab 환경이 아니므로, Google Drive 연동을 건너뜁니다. 캐시 파일 등은 현재 디렉토리에 저장됩니다.")
    data_dir = "./"


from openai import OpenAI
from getpass import getpass

if 'OPENAI_API_KEY' not in os.environ:
    try:
        api_key = getpass("OpenAI API 키를 입력하세요: ")
        os.environ['OPENAI_API_KEY'] = api_key
        print("OpenAI API 키가 설정되었습니다.")
    except Exception as e:
        print(f"API 키 입력 중 오류 발생: {e}. 환경 변수 OPENAI_API_KEY를 직접 설정해주세요.")
else:
    print("환경 변수에서 OpenAI API 키를 사용합니다.")

try:
    client = OpenAI()
except Exception as e:
    print(f"OpenAI 클라이언트 초기화 실패: {e}. API 키 설정을 확인하세요.")
    client = None

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {DEVICE}")

Mounted at /content/drive
OpenAI API 키를 입력하세요: ··········
OpenAI API 키가 설정되었습니다.
사용 디바이스: cuda


In [ ]:
#Python 3.11.12

RANDOM_SEED = 42

DECISION_BOUNDARY_THRESHOLD = 0.2 #0.2
N_SAMPLES_FOR_PROCESSING = 4000
TOP_K_RAG = 10
LM_VERBOSE = True

IMPOSSIBLE_SAMPLE_WEIGHT = 0.05
FILTERING_CONST = -0.2

# LLM 설정
LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.5
LLM_MAX_TOKENS_RESPONSE = 1000
IMPOSSIBLE_SAMPLES_CACHE_FILE = os.path.join(data_dir, "impossible_samples_cache_3.jsonl")


ANN_EPOCHS = 50
ANN_LR = 2e-4
ANN_BATCH_SIZE = 64
ANN_PATIENCE = 10

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

##function

In [ ]:
def load_adult_income_dataset():
    column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    df = pd.read_csv(url, header=None, names=column_names, na_values=" ?", skipinitialspace=True)
    df = df.dropna().reset_index(drop=True)
    df['income'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)
    return df

In [ ]:
def create_chat_completion(system_input, user_input, model=LLM_MODEL, temperature=LLM_TEMPERATURE, max_tokens=LLM_MAX_TOKENS_RESPONSE):
    if client is None: return "LLM 호출 오류: OpenAI 클라이언트가 초기화되지 않았습니다."
    messages = [{"role": "system", "content": system_input}, {"role": "user", "content": user_input}]
    response = client.chat.completions.create(model=model, messages=messages, temperature=temperature, max_tokens=max_tokens)
    return response.choices[0].message.content

In [ ]:
def create_chat_completions_parallel(requests_with_metadata, max_workers=8): # max_workers 조절
    responses_map = {}
    llm_api_requests = [req["prompt_data"] for req in requests_with_metadata if req.get("prompt_data")]
    anchor_ids_for_api_call = [req["anchor_id"] for req in requests_with_metadata if req.get("prompt_data")]
    if not llm_api_requests: return responses_map
    if client is None:
        for anchor_id in anchor_ids_for_api_call: responses_map[anchor_id] = "LLM 호출 오류: OpenAI 클라이언트가 초기화되지 않았습니다."
        return responses_map
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_anchor_id = { executor.submit( create_chat_completion, req_data["system_input"], req_data["user_input"], req_data.get("model", LLM_MODEL), req_data.get("temperature", LLM_TEMPERATURE), req_data.get("max_tokens", LLM_MAX_TOKENS_RESPONSE) ): anchor_id for req_data, anchor_id in zip(llm_api_requests, anchor_ids_for_api_call) }
        pbar = tqdm(total=len(llm_api_requests), desc="LLM 병렬 호출 (불가능 샘플)")
        for future in as_completed(future_to_anchor_id):
            anchor_id = future_to_anchor_id[future]
            try: responses_map[anchor_id] = future.result()
            except Exception as e: responses_map[anchor_id] = f"LLM 호출 오류: {str(e)}"; print(f"LLM 호출 오류 (앵커 ID {anchor_id}): {e}")
            pbar.update(1)
        pbar.close()
    return responses_map

In [ ]:
def preprocess_data(df_input):
    print("데이터 전처리 시작")
    df = df_input.copy()
    if 'income' not in df.columns: raise ValueError("'income' 컬럼이 DataFrame에 존재하지 않습니다.")
    X = df.drop('income', axis=1)
    y = df['income']
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    if 'original_index' in numerical_features:
        numerical_features.remove('original_index')
    categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

    print(f"수치형 컬럼: {numerical_features}")
    print(f"범주형 컬럼: {categorical_features}")

    numerical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')), # 평균값으로 결측치 대체
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')), # 최빈값으로 결측치 대체
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features),
            ('cat', categorical_pipeline, categorical_features)
        ],
        remainder='drop'
    )

    X_processed_array = preprocessor.fit_transform(X)

    processed_num_feature_names = numerical_features
    processed_cat_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
    processed_feature_names = processed_num_feature_names + processed_cat_feature_names

    X_processed_df = pd.DataFrame(X_processed_array, columns=processed_feature_names, index=X.index)
    print("데이터 전처리 완료")
    return X_processed_df, y, preprocessor, numerical_features, categorical_features

In [ ]:
def select_anchors_near_boundary(X_processed_df, y_series, threshold=DECISION_BOUNDARY_THRESHOLD):
    print(f"결정 경계 근처 (확률 0.5±{threshold}) 앵커 선택 중...")
    model_simple = LogisticRegression(solver='liblinear', random_state=RANDOM_SEED, max_iter=1000)
    model_simple.fit(X_processed_df, y_series)
    probabilities = model_simple.predict_proba(X_processed_df)[:, 1]
    lower_bound = 0.5 - threshold; upper_bound = 0.5 + threshold
    near_boundary_indices = X_processed_df[(probabilities > lower_bound) & (probabilities < upper_bound)].index
    print(f"총 {len(X_processed_df)}개 샘플 중 {len(near_boundary_indices)}개의 결정 경계 근처 앵커 선택됨.")
    return near_boundary_indices, model_simple

In [ ]:
def row_to_string(row_series, numerical_features, categorical_features):
    parts = []
    for col in numerical_features:
        if col in row_series: parts.append(f"{col} is {row_series[col]:.2f}")
    for col in categorical_features:
        if col in row_series: parts.append(f"{col} is {row_series[col]}")
    return "; ".join(parts)

In [ ]:
class SimpleRAGIndex: # RAG 인덱스
    def __init__(self, df_original_for_rag, numerical_features, categorical_features, sentence_model_name='all-MiniLM-L6-v2'):
        print("RAG 인덱스 초기화 중...")
        self.df_original = df_original_for_rag.copy()
        self.numerical_features = [col for col in numerical_features if col in self.df_original.columns]
        self.categorical_features = [col for col in categorical_features if col in self.df_original.columns]
        self.row_strings = self.df_original.apply(lambda r: row_to_string(r, self.numerical_features, self.categorical_features), axis=1).tolist()
        self.embedder = SentenceTransformer(sentence_model_name, device=DEVICE)
        print(f"RAG용 행 문자열 {len(self.row_strings)}개 임베딩 중...")
        self.row_embeddings = self.embedder.encode(self.row_strings, show_progress_bar=True, device=DEVICE)
        self.nn_model = NearestNeighbors(n_neighbors=max(10, TOP_K_RAG + 5), metric='cosine', algorithm='brute')
        self.nn_model.fit(self.row_embeddings); print("RAG 인덱스 초기화 완료.")
    def retrieve_similar_rows_info(self, query_row_original_series, k=TOP_K_RAG):
        query_string = row_to_string(query_row_original_series, self.numerical_features, self.categorical_features)
        query_embedding = self.embedder.encode([query_string], device=DEVICE)
        distances, indices = self.nn_model.kneighbors(query_embedding, n_neighbors=k + 1)
        similar_rows_info = []; query_row_name = query_row_original_series.name
        for i in range(indices.shape[1]):
            idx_in_rag_df = indices[0, i]; retrieved_row_original_name = self.df_original.index[idx_in_rag_df]
            if retrieved_row_original_name == query_row_name: continue
            retrieved_row_original = self.df_original.iloc[idx_in_rag_df]
            retrieved_row_string = self.row_strings[idx_in_rag_df]
            similar_rows_info.append({"original_data": retrieved_row_original.to_dict(), "string_representation": retrieved_row_string, "similarity": 1 - distances[0, i] })
            if len(similar_rows_info) >= k: break
        return similar_rows_info

In [ ]:
FEATURE_UNITS = {
    "age": "years",
    "hours-per-week": "hours",
    "capital-gain": "$",
    "capital-loss": "$",
    "education-num": "levels (numeric representation of education)",
    "fnlwgt": "(census weight, no specific unit)"
}

def build_prompt_context(similar_rows_info_list, numerical_features=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'], categorical_features=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']):
    """
    LLM 프롬프트용 컨텍스트 문자열을 구성합니다.
    각 유사 사례의 수치형 특징에 단위를 포함하고, 소득 등급 정보도 포함합니다.
    numerical_features와 categorical_features 리스트를 인자로 받도록 수정되었습니다.
    """
    context_str = "Reference context (similar real-life case):\n"
    if not similar_rows_info_list:
        context_str += "I couldn't find any similar cases to reference for context..\n"
    else:
        for i, info in enumerate(similar_rows_info_list):
            row_data = info['original_data']
            parts = []

            for col in numerical_features:
                if col in row_data:
                    value = row_data[col]
                    unit_description = FEATURE_UNITS.get(col)
                    formatted_value = ""

                    # 특징별 값 포맷팅
                    if col == "age":
                        formatted_value = str(int(value))
                    elif col == "hours-per-week":
                        formatted_value = str(int(value))
                    elif col in ["capital-gain", "capital-loss"]:
                        formatted_value = f"{float(value):.2f}"
                    elif col == "education-num":
                        formatted_value = str(int(value))
                    elif col == "fnlwgt":
                        formatted_value = str(int(value))
                    else:
                        # FEATURE_UNITS에 없고, 위에서 특별히 처리되지 않은 기타 수치형 변수
                        try:
                            formatted_value = f"{float(value):.2f}"
                        except ValueError:
                            formatted_value = str(value)
                    if unit_description:
                        if unit_description.startswith("(") and unit_description.endswith(")"):
                            parts.append(f"{col} is {formatted_value} {unit_description}")
                        else:
                            parts.append(f"{col} is {formatted_value} ({unit_description})")
                    else:
                        parts.append(f"{col} is {formatted_value}")
            for col in categorical_features:
                if col in row_data:
                    parts.append(f"{col} is {row_data[col]}")

            string_representation_with_units = "; ".join(parts)

            income_label = row_data.get('income')
            income_class_str_context = "Unknown Income"
            if income_label is not None:
                income_class_str_context = ">50K (High Income)" if income_label == 1 else "<=50K (Low Income)"

            context_str += f"  Ex {i+1}: {string_representation_with_units}; Income Class: {income_class_str_context} (Similarity: {info['similarity']:.3f})\n"

    return context_str


In [ ]:
def row_to_string_with_units(row_series, numerical_features, categorical_features, all_feature_names):
    parts = []
    for col in all_feature_names:
        if col not in row_series:
            continue
        if col in numerical_features:
            unit_str = f" ({FEATURE_UNITS.get(col, 'numeric')})" if FEATURE_UNITS.get(col) else ""
            parts.append(f"{col} is {row_series[col]:.2f}{unit_str if col in FEATURE_UNITS and FEATURE_UNITS[col] not in ['(census weight, no specific unit)','levels (numeric representation of education)'] else '' if col in FEATURE_UNITS else ''}") # Avoid .2f for specific non-currency numerics
            if col == 'education-num':
                 parts[-1] = f"{col} is {int(row_series[col])}{unit_str}"
            elif FEATURE_UNITS.get(col) == '(census weight, no specific unit)':
                 parts[-1] = f"{col} is {int(row_series[col])}{unit_str}"

        elif col in categorical_features:
            parts.append(f"{col} is {row_series[col]}")
    return "; ".join(parts)

def generate_impossible_sample_prompt(original_row_series, anchor_target_label, context_rows_str, categorical_features_map): # LLM 프롬프트 생성
    income_class_str_anchor = ">50K (High Income)" if anchor_target_label == 1 else "<=50K (Low Income)"

    original_row_prompt_str = "Original Person's Data (characteristics subject to modification):\n"
    numerical_features_in_row = [col for col in original_row_series.index if original_row_series.dtype.kind in 'if' and col in FEATURE_UNITS] # Simple check
    categorical_features_in_row = [col for col in original_row_series.index if col not in numerical_features_in_row]


    for feature, value in original_row_series.items():
        unit_str = ""
        val_str = str(value)
        if feature in FEATURE_UNITS:
            unit_str = f" ({FEATURE_UNITS[feature]})"
            if FEATURE_UNITS[feature] in ["$", "hours", "years"]:
                try: val_str = f"{float(value):.2f}" if FEATURE_UNITS[feature] == "$" else str(value)
                except ValueError: pass
            elif FEATURE_UNITS[feature] in ["levels (numeric representation of education)", "(census weight, no specific unit)"]:
                 try: val_str = str(int(value))
                 except ValueError: pass


        original_row_prompt_str += f"  - {feature}: {val_str}{unit_str}\n"
    original_row_prompt_str += f"  - Note: This individual's current income class is '{income_class_str_anchor}'.\n"

    system_prompt = (
        "You are a domain expert in social statistics and data analysis, specializing in identifying implausible data patterns. "
        "Your task is to generate a single, new data sample. This sample should be largely similar to the provided 'Original Person' "
        "but critically modified in **2-3 key features** to make the overall profile LOGICALLY IMPOSSIBLE or HIGHLY IMPROBABLE in the real world. "
        "The impossibility should arise from the conflicting combination of these few altered features, even if individual values seem plausible. "
        "Other features should remain largely unchanged from the original unless necessary to support the impossibility. "
        "You must provide a step-by-step Chain-of-Thought (CoT) reasoning (approx. 400 words) explaining WHY the generated combination is impossible, "
        "paying close attention to the **Original Person's stated income class ('" + income_class_str_anchor + "')** and general socio-economic logic. "
        "The 'Context Cases' are real-world examples to help understand typical patterns; your generated sample must deviate from such norms in a logically flawed way. "
        "Structure your response clearly: reasoning first, then the impossible sample as a JSON object."
    )

    user_prompt = f"""
{original_row_prompt_str}
{context_rows_str}

Based on the Original Person (whose income class is '{income_class_str_anchor}') and the Context Cases (if any), please perform the following:

1.  **Chain-of-Thought Reasoning (CoT):**
    * Analyze the Original Person's data, **explicitly using their income class ('{income_class_str_anchor}') as a primary factor for identifying or creating contradictions.**
    * If Context Cases are provided, use them to understand typical profiles and how they relate to income class.
    * Identify a specific combination of 2-3 features from the 'Original Person's Data'. If these features are slightly altered, they should create a logically impossible or highly improbable scenario when viewed against the **Original Person's income class or general socio-economic principles.**
    * Explain step-by-step why this new combination is impossible/improbable.
    * Here are illustrative examples of the kind of impossible scenarios you should aim for. These examples show how specific feature combinations conflict with a stated income class:

        Example 1 (Illustrating Implausibility for a Low Income Person):
          - Stated Income Class context: '<=50K (Low Income)'
          - Impossible Feature Combination: `education-num` is 16 (levels (numeric representation of education)); `hours-per-week` is 80 (hours); `marital-status` is Never-married; `occupation` is Exec-managerial
          - Reasoning: An individual who is 'Never-married', possesses a Doctorate-level education (`education-num`: 16), works 80 hours a week as an 'Exec-managerial' is exceptionally unlikely to be in the '<=50K (Low Income)' bracket due to the high earning potential typically associated with such a profile.

        Example 2 (Illustrating Implausibility for a High Income Person):
          - Stated Income Class context: '>50K (High Income)'
          - Impossible Feature Combination: `capital-gain` is 14084 ($); `native-country` is Cuba; `workclass` is Private; `hours-per-week` is 13 (hours)
          - Reasoning: A 'Private' sector worker from 'Cuba', working only 13 hours per week, yet reporting a significant `capital-gain` of $14,084 and being in the '>50K (High Income)' class is realistically implausible given the limited work hours and typical economic conditions.

        Example 3 (Illustrating Implausibility for a Low Income Person):
          - Stated Income Class context: '<=50K (Low Income)'
          - Impossible Feature Combination: `race` is Black; `sex` is Female; `occupation` is Prof-specialty; `capital-loss` is 9999 ($)
          - Reasoning: A 'Black', 'Female' 'Prof-specialty' worker in the '<=50K (Low Income)' bracket incurring a `capital-loss` of nearly $10,000 is statistically extremely rare and financially contradictory for that income level.

        Example 4 (Illustrating Implausibility for a High Income Person):
          - Stated Income Class context: '>50K (High Income)'
          - Impossible Feature Combination: `marital-status` is Divorced; `education` is HS-grad; `capital-gain` is 99999 ($); `hours-per-week` is 10 (hours)
          - Reasoning: A 'Divorced' individual with only a 'HS-grad' education, working merely 10 hours per week, yet realizing a `capital-gain` of $99,999 and being in the '>50K (High Income)' class, is logically inconsistent with typical income generation patterns.

        Example 5 (Illustrating Implausibility for a Low Income Person):
          - Stated Income Class context: '<=50K (Low Income)'
          - Impossible Feature Combination: `age` is 18 (years); `education-num` is 15 (levels (numeric representation of education)); `occupation` is Prof-specialty; `fnlwgt` is 500000 ((census weight, no specific unit))
          - Reasoning: An 18-year-old achieving a Master's level education (`education-num`: 15) and working in a 'Prof-specialty' occupation, especially with a high `fnlwgt` (census weight), being in the '<=50K (Low Income)' bracket is highly improbable due to age and educational attainment timeframe.

2.  **Generated Impossible Sample:**
    * Provide the complete data for this new, impossible person, using all features listed in the 'Original Person's Data'.
    * Critically alter **only 2-3 key features** from the Original Person to create the impossible scenario you identified in your CoT. Keep other features as close to the original as sensible, unless changes are needed to support the core impossibility.
    * Feature names to include in the JSON output: {', '.join(original_row_series.index.tolist())}.
    * When assigning values:
        * For numerical features, use appropriate numbers (refer to original values without units as a guide).
        * For categorical features, you MUST use one of the valid categories. Valid categories for relevant features from the original person are:
    **you **must** generate JSON type output in your end of response.

"""
    for cat_col, valid_vals in categorical_features_map.items():
        if cat_col in original_row_series.index: # Ensure only relevant categorical features are listed
            user_prompt += f"          - {cat_col}: {valid_vals[:3]}{' etc.' if len(valid_vals) > 3 else ''}\n"

    user_prompt += """
    **IMPORTANT OUTPUT FORMAT (Reasoning first, then the JSON object string on a new line):**
    Chain-of-Thought Reasoning:
    [Your step-by-step reasoning here, explaining the socio-economic or logical impossibility based on the original person's profile, including their income class, and the minimal changes made.]

    Generated Impossible Sample (JSON **you **must** generate JSON type output in your end of response):
    {"feature1": "value1", "feature2": value2, ... (every value must not include any unit. all features for the impossible sample, matching the provided feature names) ...}
"""
    if LM_VERBOSE:
        print(f'---system_input--- \n\n{system_prompt}\n\n------ \n\n---user_input---\n\n{user_prompt}\n\n---')
    return {"system_input": system_prompt, "user_input": user_prompt}

In [ ]:
class BaselineANN(nn.Module):
    def __init__(self, input_dim, hidden_dim1_ratio=0.75, hidden_dim2_ratio=0.5, output_dim=1): # ANN_HIDDEN_DIM_RATIO 사용하도록 수정 가능
        super(BaselineANN, self).__init__()
        hidden_dim1 = int(input_dim * hidden_dim1_ratio)
        if hidden_dim1 <= 0: hidden_dim1 = max(1, input_dim // 2 if input_dim // 2 > 0 else output_dim)
        hidden_dim2 = int(hidden_dim1 * hidden_dim2_ratio)
        if hidden_dim2 <= 0: hidden_dim2 = max(1, hidden_dim1 // 2 if hidden_dim1 // 2 > 0 else output_dim)

        self.fc_layers = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.BatchNorm1d(hidden_dim1),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.BatchNorm1d(hidden_dim2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim2, output_dim)
        )

    def forward(self, x):
        if x.ndim == 1 and x.shape[0] > 0:
             x = x.unsqueeze(0)
        elif x.ndim == 0 or (x.ndim > 0 and x.shape[0] == 0) :
            out_features = self.fc_layers[-1].out_features if hasattr(self.fc_layers[-1], 'out_features') else 1
            return torch.empty((0, out_features), device=x.device, dtype=x.dtype)

        logits = self.fc_layers(x)
        return logits.squeeze(-1) if self.fc_layers[-1].out_features == 1 else logits

def train_ann_model(model, train_loader, val_loader, optimizer, epochs, patience, model_name="ANN", use_sample_weights=False):
    print(f"{model_name} 학습 시작 (조기 종료 기능 포함, 샘플 가중치 적용 가능, F1 스코어 기준 최적화)...")
    best_val_f1 = 0.0
    epochs_no_improve = 0
    best_model_state = None

    criterion_bce_none = nn.BCEWithLogitsLoss(reduction='none').to(DEVICE)
    criterion_bce_mean_val = nn.BCEWithLogitsLoss().to(DEVICE) # 검증용은 가중치 없는 평균 손실

    for epoch in range(epochs):
        model.train()
        total_train_loss = 0

        for batch_data in tqdm(train_loader, desc=f"ANN 에폭 {epoch+1} [학습]", leave=False):
            if use_sample_weights:
                features, targets, weights = batch_data
                weights = weights.to(DEVICE)
            else:
                features, targets = batch_data
                weights = None # 가중치 사용 안 함

            features, targets = features.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(features)

            per_sample_loss = criterion_bce_none(outputs, targets)

            if use_sample_weights and weights is not None:
                loss = (per_sample_loss * weights).mean()
            else:
                loss = per_sample_loss.mean()

            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader) if len(train_loader) > 0 else 0

        # 검증 단계
        model.eval()
        all_val_preds_proba = []
        all_val_preds_binary = []
        all_val_targets = []
        total_val_loss_epoch = 0

        current_val_f1 = 0.0
        avg_val_loss = float('inf')


        if val_loader and len(val_loader) > 0:
            with torch.no_grad():
                for features_val, targets_val in tqdm(val_loader, desc=f"ANN 에폭 {epoch+1} [검증]", leave=False):
                    features_val, targets_val = features_val.to(DEVICE), targets_val.to(DEVICE)
                    outputs_val = model(features_val)
                    loss_val = criterion_bce_mean_val(outputs_val, targets_val)
                    total_val_loss_epoch += loss_val.item()

                    val_preds_proba_batch = torch.sigmoid(outputs_val).cpu().numpy()
                    all_val_preds_proba.extend(val_preds_proba_batch)
                    all_val_preds_binary.extend((val_preds_proba_batch > 0.5).astype(int)) # F1 계산용 이진 예측
                    all_val_targets.extend(targets_val.cpu().numpy())

            avg_val_loss = total_val_loss_epoch / len(val_loader)

            # 검증 F1 스코어 계산
            if len(all_val_targets) > 0: # 검증 데이터가 있는 경우에만 F1 계산
                try:
                    current_val_f1 = f1_score(all_val_targets, all_val_preds_binary, zero_division=0)
                except Exception as e:
                    print(f"경고: {model_name} 검증 F1 스코어 계산 중 오류: {e}. F1은 0.0으로 설정됨.")
                    current_val_f1 = 0.0
            else: # 검증 데이터가 없는 경우
                current_val_f1 = 0.0

            val_auc_display = 0.0
            if len(all_val_targets) > 0 and len(np.unique(all_val_targets)) > 1:
                 try: val_auc_display = roc_auc_score(all_val_targets, all_val_preds_proba)
                 except ValueError: pass


            print(f"ANN 에폭 {epoch+1} - 학습 손실{' (가중 적용)' if use_sample_weights else ''}: {avg_train_loss:.4f}, 검증 손실: {avg_val_loss:.4f}, 검증 F1: {current_val_f1:.4f}, (참고용 검증 AUC: {val_auc_display:.4f})")

            if current_val_f1 > best_val_f1:
                best_val_f1 = current_val_f1
                best_model_state = model.state_dict().copy()
                epochs_no_improve = 0
                print(f"ANN 검증 F1 스코어 개선됨. 현재 최고 F1: {best_val_f1:.4f}")
            else:
                epochs_no_improve += 1
                print(f"ANN 검증 F1 스코어 개선되지 않음. ({epochs_no_improve}/{patience})")

            if epochs_no_improve >= patience:
                print(f"{patience} 에폭 동안 ANN 검증 F1 스코어 개선 없어 조기 종료합니다.")
                break
        else:
            print(f"ANN 에폭 {epoch+1} - 학습 손실{' (가중 적용)' if use_sample_weights else ''}: {avg_train_loss:.4f} (검증 데이터 없음)")
            best_model_state = model.state_dict().copy()

    print(f"{model_name} 학습 완료.")
    if best_model_state:
        print(f"가장 좋았던 검증 F1 ({best_val_f1:.4f}) 시점의 모델로 복원합니다.")
        model.load_state_dict(best_model_state)
    else:
        print(f"경고: {model_name} 최적 모델 상태가 기록되지 않았습니다. 현재 모델 상태(마지막 에폭)를 사용합니다.")
    return model

In [ ]:
def xgb_f1_metric(preds_proba, dtrain):
    actuals = dtrain.get_label(); preds_binary = (preds_proba > 0.5).astype(int)
    f1 = f1_score(actuals, preds_binary, zero_division=0); return 'f1', f1

def evaluate_classifier(model, X_test_data, y_test_data, model_name="Classifier"): # 모델 평가
    is_xgboost_model = isinstance(model, xgb.XGBModel)
    if not is_xgboost_model and hasattr(model, 'eval') and callable(model.eval):
        model.eval();
        with torch.no_grad():
            if isinstance(X_test_data, pd.DataFrame): X_test_tensor = torch.tensor(X_test_data.values, dtype=torch.float32).to(DEVICE)
            elif isinstance(X_test_data, np.ndarray): X_test_tensor = torch.tensor(X_test_data, dtype=torch.float32).to(DEVICE)
            else: X_test_tensor = X_test_data.to(DEVICE)
            outputs = model(X_test_tensor);
            if isinstance(outputs, tuple): logits = outputs[1]
            else: logits = outputs
            if logits.ndim > 1 and logits.shape[1] > 1: y_pred_proba = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
            else: y_pred_proba = torch.sigmoid(logits).cpu().numpy().flatten()
            y_pred = (y_pred_proba > 0.5).astype(int)
    elif is_xgboost_model:
        y_pred_proba = model.predict_proba(X_test_data)[:, 1]; y_pred = model.predict(X_test_data)
    else: raise TypeError(f"지원되지 않는 모델 타입입니다: {type(model)}")
    accuracy = accuracy_score(y_test_data, y_pred)
    try: auc = roc_auc_score(y_test_data, y_pred_proba)
    except ValueError: auc = float('nan'); print(f"경고: {model_name} AUC 계산 중 오류. AUC는 NaN으로 설정됨.")
    f1 = f1_score(y_test_data, y_pred, zero_division=0)
    precision = precision_score(y_test_data, y_pred, zero_division=0)
    recall = recall_score(y_test_data, y_pred, zero_division=0)
    print(f"--- {model_name} 평가 결과 ---"); print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    return {"AUC": auc, "F1": f1, "Accuracy": accuracy}

In [ ]:
def umap_plot(
    X_train_p,
    y_train_main,
    X_impossible_p_list,
    y_impossible_opposite_label_list,
    feature_columns=None,
):
    import os
    import umap
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    umap_random_state = RANDOM_SEED
    print("\nUMAP 시각화 준비 중 (학습 데이터 및 증강 샘플)...")

    X_train_p_rich = X_train_p[y_train_main == 1]
    X_train_p_non_rich = X_train_p[y_train_main == 0]

    X_aug_rich_plot_list = []
    X_aug_non_rich_plot_list = []

    if X_impossible_p_list and y_impossible_opposite_label_list and \
       len(X_impossible_p_list) == len(y_impossible_opposite_label_list):
        print("UMAP용 증강 샘플 분류 중...")
        for i in range(len(X_impossible_p_list)):
            features = X_impossible_p_list[i]
            anchor_label = 1 - y_impossible_opposite_label_list[i]
            if anchor_label == 0:
                X_aug_rich_plot_list.append(features)
            else:
                X_aug_non_rich_plot_list.append(features)
    else:
        print("Warning: Data for augmented samples is incomplete.")

    feature_columns = X_train_p.columns
    X_aug_rich_df = pd.DataFrame(X_aug_rich_plot_list, columns=feature_columns) if X_aug_rich_plot_list else pd.DataFrame(columns=feature_columns)
    X_aug_non_rich_df = pd.DataFrame(X_aug_non_rich_plot_list, columns=feature_columns) if X_aug_non_rich_plot_list else pd.DataFrame(columns=feature_columns)

    def get_vals(data):
        if isinstance(data, pd.DataFrame): return data.values
        if isinstance(data, np.ndarray): return data
        if isinstance(data, list) and data and isinstance(data[0], np.ndarray): return np.array(data)
        return np.array([])

    orig_rich = get_vals(X_train_p_rich)
    orig_non = get_vals(X_train_p_non_rich)
    aug_rich = get_vals(X_aug_rich_df)
    aug_non = get_vals(X_aug_non_rich_df)

    X_list, labels = [], []
    if orig_rich.size:
        X_list.append(orig_rich); labels += ['Original Rich'] * len(orig_rich)
    if orig_non.size:
        X_list.append(orig_non); labels += ['Original Non-Rich'] * len(orig_non)
    if aug_rich.size:
        X_list.append(aug_rich); labels += ['Augmented Rich'] * len(aug_rich)
    if aug_non.size:
        X_list.append(aug_non); labels += ['Augmented Non-Rich'] * len(aug_non)

    if not X_list:
        print("UMAP 시각화를 위한 데이터가 없습니다.")
        return

    data_all = np.vstack(X_list)
    print(f"UMAP 학습을 위한 총 데이터 수: {len(data_all)}")

    try:
        reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2,
                            metric='euclidean', random_state=umap_random_state)
        embedding = reducer.fit_transform(data_all)
        print("UMAP 변환 완료.")
    except ImportError:
        print("UMAP 라이브러리가 설치되어 있지 않습니다.")
        return
    except Exception as e:
        print(f"UMAP 변환 중 오류 발생: {e}")
        return

    FIG_SIZE = (16, 12)
    SAVE_DIR = globals().get('data_dir', '.')

    plot_config = {
        'Original Rich': {'color': 'red', 'marker': 'o', 'alpha': 0.2, 's': 3},
        'Original Non-Rich': {'color': 'blue', 'marker': 'o', 'alpha': 0.2, 's': 3},
        'Augmented Rich': {'color': 'orange', 'marker': 'o', 'alpha': 1.0, 's': 2},
        'Augmented Non-Rich': {'color': 'cyan', 'marker': 'o', 'alpha': 1.0, 's': 2}
    }

    # Plot 1: 모든 데이터
    plt.figure(figsize=FIG_SIZE)
    for lbl, cfg in plot_config.items():
        idx = [i for i, v in enumerate(labels) if v == lbl]
        if not idx: continue
        plt.scatter(
            embedding[idx, 0], embedding[idx, 1],
            c=cfg['color'], marker=cfg['marker'],
            alpha=cfg['alpha'], s=cfg['s']
        )
    plt.axis('off')
    path_all = os.path.join(SAVE_DIR, "umap_all.png")
    plt.savefig(path_all, bbox_inches='tight')
    plt.close()
    print(f"Saved UMAP plot (all) at: {path_all}")

    # Plot 2: Original + Augmented Rich
    plt.figure(figsize=FIG_SIZE)
    for lbl in ['Original Rich', 'Original Non-Rich', 'Augmented Rich']:
        cfg = plot_config[lbl]
        idx = [i for i, v in enumerate(labels) if v == lbl]
        if not idx: continue
        plt.scatter(
            embedding[idx, 0], embedding[idx, 1],
            c=cfg['color'], marker=cfg['marker'],
            alpha=cfg['alpha'], s=cfg['s']
        )
    plt.axis('off')
    path_aug_rich = os.path.join(SAVE_DIR, "umap_augmented_rich.png")
    plt.savefig(path_aug_rich, bbox_inches='tight')
    plt.close()
    print(f"Saved UMAP plot (augmented rich) at: {path_aug_rich}")

    # Plot 3: Original only
    plt.figure(figsize=FIG_SIZE)
    for lbl in ['Original Rich', 'Original Non-Rich']:
        cfg = plot_config[lbl]
        idx = [i for i, v in enumerate(labels) if v == lbl]
        if not idx: continue
        plt.scatter(
            embedding[idx, 0], embedding[idx, 1],
            c=cfg['color'], marker=cfg['marker'],
            alpha=cfg['alpha'], s=cfg['s']
        )
    plt.axis('off')
    path_orig = os.path.join(SAVE_DIR, "umap_original_only.png")
    plt.savefig(path_orig, bbox_inches='tight')
    plt.close()
    print(f"Saved UMAP plot (original only) at: {path_orig}")

    print("\nUMAP 시각화 준비 중 (학습 데이터만으로 재학습)...")
    try:
        reducer_train_only = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2,
                                       metric='euclidean', random_state=umap_random_state)
        embedding_train_only = reducer_train_only.fit_transform(X_train_p.values)
        print("학습 데이터만으로 UMAP 변환 완료.")

        plt.figure(figsize=FIG_SIZE)
        labels_train_only = ['Original Rich' if l == 1 else 'Original Non-Rich' for l in y_train_main]

        for lbl in ['Original Rich', 'Original Non-Rich']:
            cfg = plot_config[lbl]
            idx = [i for i, v in enumerate(labels_train_only) if v == lbl]
            if not idx: continue

            plt.scatter(
                embedding_train_only[idx, 0], embedding_train_only[idx, 1],
                c=cfg['color'], marker=cfg['marker'],
                alpha=cfg['alpha'], s=cfg['s']
            )

        plt.axis('off')
        path_train_fit = os.path.join(SAVE_DIR, "umap_training_data_fit_only.png")
        plt.savefig(path_train_fit, bbox_inches='tight')
        plt.close()
        print(f"Saved UMAP plot (fit on training data only) at: {path_train_fit}")
    except Exception as e:
        print(f"학습 데이터만으로 UMAP 시각화 중 오류 발생: {e}")

In [ ]:
def parse_llm_response_for_impossible_sample(response_str, expected_feature_names_original_order): # LLM 응답 파싱
    if LM_VERBOSE:
        print(f"\n--- Raw LLM Response ---")
        print(response_str)
        print("--------------------------------------")
    try:
        parts = response_str.split("Generated Impossible Sample (JSON):", 1)
        if len(parts) < 2:
            reasoning_part = response_str.split("Chain-of-Thought Reasoning:", 1)[-1].strip() if "Chain-of-Thought Reasoning:" in response_str else "추론 과정이 명확히 분리되지 않았습니다."
            json_part_str = parts[0]
        else:
            reasoning_part = parts[0].replace("Chain-of-Thought Reasoning:", "").strip()
            json_part_str = parts[1].strip()
        first_brace = json_part_str.find('{'); last_brace = json_part_str.rfind('}')
        if first_brace != -1 and last_brace != -1 and last_brace > first_brace: json_data_str_cleaned = json_part_str[first_brace : last_brace+1]
        else: raise json.JSONDecodeError("응답의 뒷부분에서 유효한 JSON 객체를 찾을 수 없습니다.", json_part_str, 0)
        impossible_sample_dict_from_llm = json.loads(json_data_str_cleaned)
        parsed_sample_data_ordered = { fname: impossible_sample_dict_from_llm.get(fname) for fname in expected_feature_names_original_order }
        return pd.Series(parsed_sample_data_ordered, index=expected_feature_names_original_order), reasoning_part
    except Exception as e:
        print(f"LLM 응답 파싱 오류: {e}\n응답 앞부분: {response_str[:300]}...\n응답 뒷부분 (JSON 예상): ...{json_part_str[-300:] if 'json_part_str' in locals() else ''}")
        return None, None

def load_impossible_samples_cache(cache_file=IMPOSSIBLE_SAMPLES_CACHE_FILE): # 캐시 로드
    cache = {}
    if os.path.exists(cache_file):
        print(f"캐시 파일 로드 중: {cache_file}")
        with open(cache_file, 'r', encoding='utf-8') as f:
            for line_idx, line in enumerate(f):
                try: data = json.loads(line); cache[int(data["anchor_id"])] = (data["impossible_sample_dict"], data["reasoning"])
                except json.JSONDecodeError as je: print(f"캐시 파일 JSON 파싱 오류 (라인 {line_idx+1} 무시): {je} - 내용: {line.strip()}")
                except Exception as ex: print(f"캐시 파일 로드 중 알 수 없는 오류 (라인 {line_idx+1} 무시): {ex} - 내용: {line.strip()}")
    else: print(f"캐시 파일({cache_file})이 존재하지 않습니다. 새로 생성됩니다.")
    print(f"{len(cache)}개의 불가능 샘플을 캐시에서 로드함.")
    return cache

def save_impossible_sample_to_cache(anchor_id, impossible_sample_dict, reasoning, cache_file=IMPOSSIBLE_SAMPLES_CACHE_FILE): # 캐시 저장
    cache_dir = os.path.dirname(cache_file);
    if cache_dir and not os.path.exists(cache_dir):
        try: os.makedirs(cache_dir); print(f"캐시 디렉토리 생성: {cache_dir}")
        except OSError as e: print(f"캐시 디렉토리 생성 실패: {e}. 현재 디렉토리에 저장 시도."); cache_file = os.path.basename(cache_file)
    with open(cache_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps({"anchor_id": int(anchor_id), "impossible_sample_dict": impossible_sample_dict, "reasoning": reasoning}) + "\n")

In [ ]:
def main(is_umap):
    df_original_full = load_adult_income_dataset()
    X_processed_full, y_full, preprocessor, numerical_f, categorical_f = preprocess_data(df_original_full.copy())
    original_feature_names_for_impossible_sample = [col for col in df_original_full.columns if col not in ['income', 'original_index']]

    X_train_val_idx, X_test_idx, y_train_val, y_test = train_test_split(df_original_full.index, y_full, test_size=0.2, random_state=RANDOM_SEED, stratify=y_full)
    X_train_idx, X_val_idx, y_train_main, y_val_main = train_test_split(X_train_val_idx, y_train_val, test_size=0.2, random_state=RANDOM_SEED, stratify=y_train_val)

    df_original_train = df_original_full.loc[X_train_idx]
    df_original_test = df_original_full.loc[X_test_idx]
    X_train_p = X_processed_full.loc[X_train_idx]
    X_val_p = X_processed_full.loc[X_val_idx]
    X_test_p = X_processed_full.loc[X_test_idx]

    print(f"최종 학습 데이터 (전처리됨): {X_train_p.shape}, 레이블: {len(y_train_main)}")
    print(f"최종 검증 데이터 (전처리됨): {X_val_p.shape}, 레이블: {len(y_val_main)}")
    print(f"최종 테스트 데이터 (전처리됨): {X_test_p.shape}, 레이블: {len(y_test)}")

    selected_anchor_indices, model_simple_for_filtering = select_anchors_near_boundary(X_train_p, y_train_main)
    if N_SAMPLES_FOR_PROCESSING is not None and N_SAMPLES_FOR_PROCESSING < len(selected_anchor_indices):
        final_selected_anchor_indices = np.random.choice(selected_anchor_indices, N_SAMPLES_FOR_PROCESSING, replace=False)
    else: final_selected_anchor_indices = selected_anchor_indices
    print(f"불가능 샘플 생성을 위한 최종 앵커 수: {len(final_selected_anchor_indices)}")
    df_anchors_original = df_original_train.loc[final_selected_anchor_indices]

    categorical_features_map_for_llm = { col: df_original_train[col].unique().tolist() for col in categorical_f }

    impossible_samples_cache = load_impossible_samples_cache()
    all_anchors_fully_cached = True
    if df_anchors_original.empty: print("선택된 앵커가 없어 불가능 샘플 생성 및 RAG 인덱스 초기화를 건너뜁니다.")
    elif not impossible_samples_cache: print("불가능 샘플 캐시가 비어있어 RAG 인덱스를 초기화하고 LLM 호출이 필요합니다."); all_anchors_fully_cached = False
    else:
        for anchor_id in df_anchors_original.index:
            if int(anchor_id) not in impossible_samples_cache: all_anchors_fully_cached = False; break
    rag_index = None
    if not all_anchors_fully_cached and not df_anchors_original.empty:
        print("캐시에 없는 불가능 샘플이 있어 RAG 인덱스를 초기화합니다..."); rag_index = SimpleRAGIndex(df_original_train, numerical_f, categorical_f)
    else: print("모든 필수 불가능 샘플이 캐시에 존재하거나 처리할 앵커가 없으므로 RAG 인덱스 초기화를 건너뜁니다.")

    print(f"\n{len(df_anchors_original)}개의 선택된 앵커에 대해 불가능 샘플 생성 시작 (캐싱 및 병렬 처리)...")
    llm_requests_to_make = []
    if not df_anchors_original.empty:
        for anchor_id, anchor_original_series in tqdm(df_anchors_original.iterrows(), total=len(df_anchors_original), desc="앵커용 LLM 요청 준비"):
            if int(anchor_id) not in impossible_samples_cache:
                context_str = "RAG 컨텍스트를 가져올 수 없음 (또는 RAG 비활성화)."
                if rag_index: similar_rows = rag_index.retrieve_similar_rows_info(anchor_original_series, k=TOP_K_RAG); context_str = build_prompt_context(similar_rows)
                else: print(f"경고: 앵커 ID {anchor_id} RAG 인덱스 없음. 컨텍스트 없이 진행.")
                prompt_data = generate_impossible_sample_prompt(anchor_original_series.drop('original_index', errors='ignore'), y_train_main.loc[anchor_id], context_str, categorical_features_map_for_llm)
                llm_requests_to_make.append({"anchor_id": int(anchor_id), "prompt_data": prompt_data})
    if llm_requests_to_make:
        print(f"{len(llm_requests_to_make)}개의 새로운 불가능 샘플에 대해 LLM 호출 실행..."); llm_generated_responses_map = create_chat_completions_parallel(llm_requests_to_make)
        for anchor_id, response_str in tqdm(llm_generated_responses_map.items(), desc="LLM 응답 파싱 및 캐시 저장"):
            if "LLM 호출 오류" in response_str: print(f"앵커 ID {anchor_id} LLM 오류: {response_str}"); continue
            parsed_series, reasoning = parse_llm_response_for_impossible_sample(response_str, original_feature_names_for_impossible_sample)
            if parsed_series is not None: impossible_samples_cache[anchor_id] = (parsed_series.to_dict(), reasoning); save_impossible_sample_to_cache(anchor_id, parsed_series.to_dict(), reasoning)
            else: print(f"앵커 ID {anchor_id} LLM 응답 파싱 실패.")
    elif not df_anchors_original.empty: print("LLM 호출이 필요한 새로운 불가능 샘플 없음 (모두 캐시되었거나 처리할 앵커 없음).")

    X_impossible_p_list = []; y_impossible_opposite_label_list = []
    print(f"\n총 {len(final_selected_anchor_indices)}개의 앵커에 대해 증강 데이터셋 구성 시도 (필터링 포함)...")
    for anchor_id in tqdm(final_selected_anchor_indices, desc="증강 데이터용 불가능 샘플 처리 및 필터링"):
        if anchor_id in impossible_samples_cache:
            impossible_sample_dict_cached, _ = impossible_samples_cache[anchor_id]
            impossible_series_original = pd.Series(impossible_sample_dict_cached).reindex(original_feature_names_for_impossible_sample)
        else: continue
        anchor_target = y_train_main.loc[anchor_id]
        try:
            impossible_df_for_transform_filter = pd.DataFrame([impossible_series_original.to_dict()], columns=original_feature_names_for_impossible_sample)
            df_to_transform_correct_order_filter = impossible_df_for_transform_filter[numerical_f + categorical_f]
            impossible_sample_processed_filter = preprocessor.transform(df_to_transform_correct_order_filter)
        except Exception as e: print(f"필터링용 불가능 샘플 전처리 오류 (앵커 ID {anchor_id}): {e}"); continue
        prob_class1_for_impossible = model_simple_for_filtering.predict_proba(impossible_sample_processed_filter)[:, 1][0]
        keep_sample = False
        if anchor_target == 0: # 원본이 0 (새로 부여될 레이블은 1) -> model_simple이 1으로 예측하는 경향이 있을 때 유지
            if prob_class1_for_impossible > (0.5 - FILTERING_CONST): keep_sample = True
        elif anchor_target == 1: # 원본이 1 (새로 부여될 레이블은 0) -> model_simple이 0로 예측하는 경향이 있을 때 유지
            if prob_class1_for_impossible < (0.5 + FILTERING_CONST): keep_sample = False

        if not keep_sample: continue
        X_impossible_p_list.append(impossible_sample_processed_filter[0]); y_impossible_opposite_label_list.append(1 - anchor_target)
    print(f"필터링 후, 총 {len(X_impossible_p_list)}개의 유효한 불가능 샘플을 증강에 사용합니다.")
    impossible_labels_series = pd.Series(y_impossible_opposite_label_list)
    label_counts = impossible_labels_series.value_counts()

    count_as_high_income = label_counts.get(1, 0) # 새로 '고소득(1)' 레이블이 부여된 불가능 샘플 수
    count_as_low_income = label_counts.get(0, 0)  # 새로 '저소득(0)' 레이블이 부여된 불가능 샘플 수

    print(f"  - \"고소득층(>50K)으로 레이블된 불가능 샘플\" (원래 앵커는 저소득층): {count_as_high_income} 건")
    print(f"  - \"저소득층(<=50K)으로 레이블된 불가능 샘플\" (원래 앵커는 고소득층): {count_as_low_income} 건")
    print("================================================")
    print(f"총 채택된 불가능 샘플 수: {len(y_impossible_opposite_label_list)} 건")

    if X_impossible_p_list:
        X_impossible_df = pd.DataFrame(X_impossible_p_list, columns=X_train_p.columns)
        X_train_p_aug = pd.concat([X_train_p, X_impossible_df], ignore_index=True)
        y_train_aug = pd.concat([y_train_main, pd.Series(y_impossible_opposite_label_list)], ignore_index=True)
        sample_weights_train_aug = np.array([1.0] * len(y_train_main) + [IMPOSSIBLE_SAMPLE_WEIGHT] * len(y_impossible_opposite_label_list))
        print(f"증강된 학습 데이터 크기: {X_train_p_aug.shape}, 증강된 레이블 크기: {len(y_train_aug)}")
    else:
        print("증강할 불가능 샘플이 없어 원본 데이터만 사용합니다."); X_train_p_aug = X_train_p.copy(); y_train_aug = y_train_main.copy(); sample_weights_train_aug = np.ones(len(y_train_aug))

    print("\n[베이스라인 1] XGBoost (원본 데이터, F1 기준 조기 종료, 네이티브 API) 학습 중...")

    # 데이터를 XGBoost DMatrix 형태로 변환
    try:
        dtrain_baseline = xgb.DMatrix(X_train_p, label=y_train_main)
        dval_baseline = xgb.DMatrix(X_val_p, label=y_val_main)
        dtest_baseline = xgb.DMatrix(X_test_p, label=y_test)
    except Exception as e:
        print(f"DMatrix 변환 중 오류: {e}")
        baseline_xgb_metrics = {"AUC": float('nan'), "F1": float('nan'), "Accuracy": float('nan')}

    if 'dtrain_baseline' in locals():
        params_xgb = {
            'objective': 'binary:logistic',
            'seed': RANDOM_SEED,
            'eta': 0.05,
        }

        # 모델 학습 (xgb.train 사용)
        watchlist_baseline = [(dtrain_baseline, 'train'), (dval_baseline, 'eval')]

        print("XGBoost 네이티브 API로 학습 시작...")
        bst_baseline_xgb = xgb.train(
            params_xgb,
            dtrain_baseline,
            num_boost_round=3000,
            evals=watchlist_baseline,
            feval=xgb_f1_metric,
            maximize=True,
            early_stopping_rounds=100,
            verbose_eval=False
        )
        y_pred_proba_baseline_xgb = bst_baseline_xgb.predict(dtest_baseline, iteration_range=(0, bst_baseline_xgb.best_iteration + 1))
        y_pred_binary_baseline_xgb = (y_pred_proba_baseline_xgb > 0.5).astype(int)

        baseline_xgb_accuracy = accuracy_score(y_test, y_pred_binary_baseline_xgb)
        try:
            baseline_xgb_auc = roc_auc_score(y_test, y_pred_proba_baseline_xgb)

        except ValueError:
            baseline_xgb_auc = float('nan')
        baseline_xgb_f1 = f1_score(y_test, y_pred_binary_baseline_xgb, zero_division=0)
        baseline_xgb_precision = precision_score(y_test, y_pred_binary_baseline_xgb, zero_division=0)
        baseline_xgb_recall = recall_score(y_test, y_pred_binary_baseline_xgb, zero_division=0)

        print(f"--- Baseline 모델 (XGBoost 네이티브) 평가 결과 ---")
        print(f"Accuracy: {baseline_xgb_accuracy:.4f}, AUC: {baseline_xgb_auc:.4f}, F1: {baseline_xgb_f1:.4f}, Precision: {baseline_xgb_precision:.4f}, Recall: {baseline_xgb_recall:.4f}")
        baseline_xgb_metrics = {"AUC": baseline_xgb_auc, "F1": baseline_xgb_f1, "Accuracy": baseline_xgb_accuracy}
    print("\n[베이스라인 2] ANN (원본 데이터) 학습 중...")
    train_loader_ann_base = DataLoader(TensorDataset(torch.tensor(X_train_p.values,dtype=torch.float32), torch.tensor(y_train_main.values,dtype=torch.float32)), batch_size=ANN_BATCH_SIZE, shuffle=True, drop_last=True)
    val_loader_ann_base = DataLoader(TensorDataset(torch.tensor(X_val_p.values,dtype=torch.float32), torch.tensor(y_val_main.values,dtype=torch.float32)), batch_size=ANN_BATCH_SIZE, shuffle=False) if len(X_val_p) > 0 else None
    model_baseline_ann = BaselineANN(input_dim=X_train_p.shape[1]).to(DEVICE)
    optimizer_ann_base = optim.Adam(model_baseline_ann.parameters(), lr=ANN_LR)
    model_baseline_ann = train_ann_model(model_baseline_ann, train_loader_ann_base, val_loader_ann_base, optimizer_ann_base, ANN_EPOCHS, ANN_PATIENCE, model_name="Baseline ANN (Original Data)", use_sample_weights=False)
    baseline_ann_metrics = evaluate_classifier(model_baseline_ann, X_test_p, y_test, "Baseline ANN (Original Data)")
    # --- SMOTE 증강 데이터 생성 ---
    print("\nSMOTE를 사용하여 학습 데이터 증강 중...")
    # 소수 클래스(레이블 1)의 현재 샘플 수
    num_minority_original = np.sum(y_train_main == 1)
    num_majority_original = np.sum(y_train_main == 0)

    num_synthetic_to_generate = int(len(y_impossible_opposite_label_list))

    desired_minority_samples_after_smote = num_minority_original + num_synthetic_to_generate


    smote_sampling_strategy = {1: desired_minority_samples_after_smote}

    try:
        smote = SMOTE(sampling_strategy=smote_sampling_strategy, random_state=RANDOM_SEED, k_neighbors=5) # k_neighbors는 소수 클래스 샘플 수보다 작아야 함
        min_samples_in_minority = np.sum(y_train_main == 1)
        if min_samples_in_minority <= smote.k_neighbors:
            smote.k_neighbors = min_samples_in_minority - 1 if min_samples_in_minority > 1 else 1
            print(f"SMOTE k_neighbors를 {smote.k_neighbors}로 조정 (소수 클래스 샘플 부족)")

        if min_samples_in_minority > 0 : # 소수 클래스 샘플이 있을 때만 SMOTE 적용
            X_train_smote, y_train_smote = smote.fit_resample(X_train_p, y_train_main)
            X_train_smote_df = pd.DataFrame(X_train_smote, columns=X_train_p.columns)
            y_train_smote_series = pd.Series(y_train_smote)
            print(f"SMOTE 증강 후 학습 데이터 크기: {X_train_smote_df.shape}, 레이블 분포:\n{y_train_smote_series.value_counts(normalize=True)}")
        else:
            print("SMOTE 적용 불가: 학습 데이터에 소수 클래스 샘플이 없습니다.")
            X_train_smote_df = X_train_p.copy() # 원본 사용
            y_train_smote_series = y_train_main.copy()

    except Exception as e:
        print(f"SMOTE 처리 중 오류 발생: {e}. 원본 학습 데이터를 사용합니다.")
        X_train_smote_df = X_train_p.copy() # 오류 시 원본 사용
        y_train_smote_series = y_train_main.copy()


    print("\n[베이스라인 3] SMOTE + XGBoost (네이티브 API, F1 기준 조기 종료) 학습 중...")

    smote_xgb_metrics = {"AUC": float('nan'), "F1": float('nan'), "Accuracy": float('nan')}

    if 'X_train_smote_df' in locals() and not X_train_smote_df.empty: # SMOTE 증강된 학습 데이터가 있는 경우
        try:
            dtrain_smote = xgb.DMatrix(X_train_smote_df, label=y_train_smote_series)

            if 'dval_main' not in locals() or dval_main is None:
                if 'X_val_p' in locals() and 'y_val_main' in locals() and not X_val_p.empty:
                    dval_main = xgb.DMatrix(X_val_p, label=y_val_main)
                    print("SMOTE + XGBoost용 원본 검증 DMatrix(dval_main) 생성됨.")
                else:
                    print("경고: SMOTE + XGBoost 조기 종료를 위한 원본 검증 데이터(X_val_p)가 없습니다. 조기 종료 없이 진행될 수 있습니다.")
            if 'dtest_baseline' not in locals() or dtest_baseline is None:
                if 'X_test_p' in locals() and 'y_test' in locals() and not X_test_p.empty:
                    dtest_baseline = xgb.DMatrix(X_test_p, label=y_test)
                    print("SMOTE + XGBoost용 원본 테스트 DMatrix(dtest_baseline) 생성됨.")
                else:
                    print("오류: SMOTE + XGBoost 평가를 위한 원본 테스트 데이터(X_test_p)가 없습니다.")
                    raise ValueError("테스트 데이터 DMatrix를 생성할 수 없습니다.")

            watchlist_smote = [(dtrain_smote, 'train_smote')]
            early_stopping_active_smote = False
            if dval_main: # 원본 검증 DMatrix가 있을 경우에만 추가 및 조기 종료 활성화
                watchlist_smote.append((dval_main, 'eval_original_val'))
                early_stopping_active_smote = True

            print("SMOTE + XGBoost 네이티브 API로 학습 시작...")
            bst_smote_xgb = xgb.train(
                params_xgb, # 이전에 정의된 params_xgb 사용 또는 params_xgb_smote 사용
                dtrain_smote,
                num_boost_round=3000,  # 최대 부스팅 라운드 수 (조절 가능)
                evals=watchlist_smote,
                feval=xgb_f1_metric,
                maximize=True,
                early_stopping_rounds=100 if early_stopping_active_smote else None,
                verbose_eval=False
            )

            best_iteration_smote = bst_smote_xgb.best_iteration if early_stopping_active_smote and bst_smote_xgb.best_iteration > 0 else bst_smote_xgb.num_boosted_rounds()

            y_pred_proba_smote_xgb = bst_smote_xgb.predict(dtest_baseline, iteration_range=(0, best_iteration_smote))
            y_pred_binary_smote_xgb = (y_pred_proba_smote_xgb > 0.5).astype(int)

            smote_xgb_accuracy = accuracy_score(y_test, y_pred_binary_smote_xgb)
            try:
                smote_xgb_auc = roc_auc_score(y_test, y_pred_proba_smote_xgb)
            except ValueError:
                smote_xgb_auc = float('nan')
            smote_xgb_f1 = f1_score(y_test, y_pred_binary_smote_xgb, zero_division=0)
            smote_xgb_precision = precision_score(y_test, y_pred_binary_smote_xgb, zero_division=0)
            smote_xgb_recall = recall_score(y_test, y_pred_binary_smote_xgb, zero_division=0)

            print(f"--- SMOTE + XGBoost (네이티브) 평가 결과 (원본 테스트셋 기준) ---")
            print(f"Accuracy: {smote_xgb_accuracy:.4f}, AUC: {smote_xgb_auc:.4f}, F1: {smote_xgb_f1:.4f}, Precision: {smote_xgb_precision:.4f}, Recall: {smote_xgb_recall:.4f}")
            smote_xgb_metrics = {"AUC": smote_xgb_auc, "F1": smote_xgb_f1, "Accuracy": smote_xgb_accuracy}

        except Exception as e:
            print(f"SMOTE + XGBoost 학습 또는 평가 중 오류 발생: {e}")
    else:
        print("SMOTE 증강된 학습 데이터가 없어 SMOTE + XGBoost 학습을 건너뜁니다.")

    print("\n[베이스라인 4] SMOTE + ANN 학습 중...")
    train_loader_ann_smote = DataLoader(
        TensorDataset(torch.tensor(X_train_smote_df.values, dtype=torch.float32),
                      torch.tensor(y_train_smote_series.values, dtype=torch.float32)),
        batch_size=ANN_BATCH_SIZE, shuffle=True, drop_last=True
    )
    val_loader_ann_smote = DataLoader(
        TensorDataset(torch.tensor(X_val_p.values, dtype=torch.float32),
                      torch.tensor(y_val_main.values, dtype=torch.float32)),
        batch_size=ANN_BATCH_SIZE, shuffle=False
    ) if len(X_val_p) > 0 else None

    model_smote_ann = BaselineANN(input_dim=X_train_smote_df.shape[1]).to(DEVICE)
    optimizer_ann_smote = optim.Adam(model_smote_ann.parameters(), lr=ANN_LR)

    model_smote_ann = train_ann_model(
        model_smote_ann, train_loader_ann_smote, val_loader_ann_smote, optimizer_ann_smote,
        ANN_EPOCHS, ANN_PATIENCE, model_name="SMOTE + ANN", use_sample_weights=False # SMOTE는 데이터 균형을 맞추므로 추가 가중치 불필요
    )
    smote_ann_metrics = evaluate_classifier(model_smote_ann, X_test_p, y_test, "SMOTE + ANN")

    print("\n[제안 모델 1] Augmented XGBoost (F1 기준 조기 종료, 네이티브 API, 원본 검증셋) 학습 중...")

    aug_xgb_metrics = {"AUC": float('nan'), "F1": float('nan'), "Accuracy": float('nan')} # 결과 초기화

    if 'X_train_p_aug' in locals() and not X_train_p_aug.empty: # 증강된 학습 데이터가 있는 경우
        try:
            dtrain_aug = xgb.DMatrix(X_train_p_aug, label=y_train_aug, weight=sample_weights_train_aug)

            if 'dval_main' not in locals() or dval_main is None: # dval_main 존재 확인
                if 'X_val_p' in locals() and 'y_val_main' in locals() and not X_val_p.empty:
                    dval_main = xgb.DMatrix(X_val_p, label=y_val_main)
                    print("Augmented XGBoost용 원본 검증 DMatrix(dval_main) 생성됨.")
                else:
                    print("경고: Augmented XGBoost 조기 종료를 위한 원본 검증 데이터(X_val_p)가 없습니다. 조기 종료 없이 진행됩니다.")
                    dval_main = None # 조기 종료 비활성화

            if 'dtest_baseline' not in locals() or dtest_baseline is None: # dtest_baseline 존재 확인
                if 'X_test_p' in locals() and 'y_test' in locals() and not X_test_p.empty:
                    dtest_baseline = xgb.DMatrix(X_test_p, label=y_test)
                    print("Augmented XGBoost용 원본 테스트 DMatrix(dtest_baseline) 생성됨.")
                else:
                    print("오류: Augmented XGBoost 평가를 위한 원본 테스트 데이터(X_test_p)가 없습니다.")
                    raise ValueError("테스트 데이터 DMatrix를 생성할 수 없습니다.")

            params_xgb_aug = {
                'objective': 'binary:logistic',
                'seed': RANDOM_SEED,
                'eta': 0.05
            }

            watchlist_aug = [(dtrain_aug, 'train_aug')]
            if dval_main:
                watchlist_aug.append((dval_main, 'eval_original_val'))

            print("Augmented XGBoost 네이티브 API로 학습 시작...")
            bst_aug_xgb = xgb.train(
                params_xgb_aug,
                dtrain_aug,
                num_boost_round=3000,
                evals=watchlist_aug,
                feval=xgb_f1_metric,
                maximize=True,
                early_stopping_rounds=100 if dval_main else None,
                verbose_eval=False
            )

            # 4. 원본 테스트 데이터로 예측 및 평가
            best_iteration_aug = bst_aug_xgb.best_iteration if dval_main and bst_aug_xgb.best_iteration > 0 else bst_aug_xgb.num_boosted_rounds()

            y_pred_proba_aug_xgb = bst_aug_xgb.predict(dtest_baseline, iteration_range=(0, best_iteration_aug))
            y_pred_binary_aug_xgb = (y_pred_proba_aug_xgb > 0.5).astype(int)

            aug_xgb_accuracy = accuracy_score(y_test, y_pred_binary_aug_xgb)
            try:
                aug_xgb_auc = roc_auc_score(y_test, y_pred_proba_aug_xgb)
            except ValueError:
                aug_xgb_auc = float('nan')
            aug_xgb_f1 = f1_score(y_test, y_pred_binary_aug_xgb, zero_division=0)
            aug_xgb_precision = precision_score(y_test, y_pred_binary_aug_xgb, zero_division=0)
            aug_xgb_recall = recall_score(y_test, y_pred_binary_aug_xgb, zero_division=0)

            print(f"--- Augmented XGBoost (네이티브) 평가 결과 (원본 테스트셋 기준) ---")
            print(f"Accuracy: {aug_xgb_accuracy:.4f}, AUC: {aug_xgb_auc:.4f}, F1: {aug_xgb_f1:.4f}, Precision: {aug_xgb_precision:.4f}, Recall: {aug_xgb_recall:.4f}")
            aug_xgb_metrics = {"AUC": aug_xgb_auc, "F1": aug_xgb_f1, "Accuracy": aug_xgb_accuracy}

        except Exception as e:
            print(f"Augmented XGBoost 학습 또는 평가 중 오류 발생: {e}")
    else:
        print("증강된 학습 데이터(X_train_p_aug)가 없어 Augmented XGBoost 학습을 건너뜁니다.")

    print("\n[제안 모델 2] Augmented ANN 학습 중 (학습:증강데이터, 검증:원본데이터)...") # 로그 메시지 수정

    # Augmented ANN용 학습 데이터로더 (증강된 전체 학습 데이터 및 가중치 사용)
    if len(X_train_p_aug) > 0:
        aug_train_dataset_ann = TensorDataset(
            torch.tensor(X_train_p_aug.values, dtype=torch.float32),
            torch.tensor(y_train_aug.values, dtype=torch.float32),
            torch.tensor(sample_weights_train_aug, dtype=torch.float32)
        )
        aug_train_loader_ann = DataLoader(aug_train_dataset_ann, batch_size=ANN_BATCH_SIZE, shuffle=True, drop_last=True)
    else: # 증강된 학습 데이터가 없는 경우 (예: 불가능 샘플이 하나도 생성/채택되지 않음)
        print("경고: Augmented ANN을 위한 증강 학습 데이터가 없습니다. 학습을 건너뜁니다.")
        aug_train_loader_ann = None # 학습 로더 없음
        aug_ann_metrics = {"AUC": float('nan'), "F1": float('nan'), "Accuracy": float('nan')}


    # Augmented ANN용 검증 데이터로더 (원본 검증 데이터 X_val_p, y_val_main 사용)
    aug_val_loader_ann = None
    if len(X_val_p) > 0 : # 원본 검증 데이터가 있는 경우
        aug_val_dataset_ann = TensorDataset(
            torch.tensor(X_val_p.values, dtype=torch.float32),
            torch.tensor(y_val_main.values, dtype=torch.float32) # 검증셋에는 가중치 불필요
        )
        aug_val_loader_ann = DataLoader(aug_val_dataset_ann, batch_size=ANN_BATCH_SIZE, shuffle=False)
    else:
        print("경고: Augmented ANN을 위한 원본 검증 데이터가 없습니다.")


    model_aug_ann = BaselineANN(input_dim=X_train_p_aug.shape[1] if not X_train_p_aug.empty else X_train_p.shape[1]).to(DEVICE)
    optimizer_ann_aug = optim.Adam(model_aug_ann.parameters(), lr=ANN_LR)

    if aug_train_loader_ann and len(aug_train_loader_ann) > 0: # 학습 로더에 데이터가 있을 때만 학습
        model_aug_ann = train_ann_model(
            model_aug_ann, aug_train_loader_ann, aug_val_loader_ann, optimizer_ann_aug,
            ANN_EPOCHS, ANN_PATIENCE, model_name="Augmented ANN", use_sample_weights=True # 가중치 사용 명시
        )
        aug_ann_metrics = evaluate_classifier(model_aug_ann, X_test_p, y_test, "Augmented ANN")
    elif 'aug_ann_metrics' not in locals(): # 학습이 안된 경우 aug_ann_metrics 초기화
        print("Augmented ANN 학습 데이터가 없어 학습 및 평가를 건너뜁니다.")
        aug_ann_metrics = {"AUC": float('nan'), "F1": float('nan'), "Accuracy": float('nan')}

    print("\n--- 최종 성능 비교 (테스트 데이터 기준) ---")
    results_list = [
        {"Model": "Baseline XGBoost", **baseline_xgb_metrics},
        {"Model": "Baseline ANN", **baseline_ann_metrics},
        {"Model": "SMOTE + XGBoost", **smote_xgb_metrics}, # SMOTE XGBoost 결과 추가
        {"Model": "SMOTE + ANN", **smote_ann_metrics},     # SMOTE ANN 결과 추가
        {"Model": "Augmented XGBoost", **aug_xgb_metrics},
        {"Model": "Augmented ANN", **aug_ann_metrics}
    ]
    results_df = pd.DataFrame(results_list)
    print(results_df)

    # --- 성능 향상률 분석 부분도 SMOTE 모델에 대한 비교를 추가하거나, 기존 방식 유지 ---
    if not results_df.empty and 'AUC' in results_df.columns and results_df[['AUC', 'F1']].notna().all(axis=None):
        results_df.set_index('Model', inplace=True)

        results_df_melted = results_df.reset_index().melt(id_vars="Model", var_name="Metric", value_name="Score")

        auc_baseline_xgb = results_df.loc['Baseline XGBoost', 'AUC']; f1_baseline_xgb = results_df.loc['Baseline XGBoost', 'F1']; acc_baseline_xgb = results_df.loc['Baseline XGBoost', 'Accuracy']
        auc_base_ann = results_df.loc['Baseline ANN', 'AUC']; f1_base_ann = results_df.loc['Baseline ANN', 'F1']; acc_base_ann = results_df.loc['Baseline ANN', 'Accuracy']

        auc_augmented_xgb = results_df.loc['Augmented XGBoost', 'AUC']; f1_augmented_xgb = results_df.loc['Augmented XGBoost', 'F1']; acc_augmented_xgb = results_df.loc['Augmented XGBoost', 'Accuracy']
        auc_imp_xgb=(auc_augmented_xgb-auc_baseline_xgb); f1_imp_xgb=(f1_augmented_xgb-f1_baseline_xgb); acc_imp_xgb=(acc_augmented_xgb-acc_baseline_xgb)
        auc_p_imp_xgb=(auc_imp_xgb/auc_baseline_xgb*100) if auc_baseline_xgb!=0 else float('inf') if auc_imp_xgb > 0 else 0.0
        f1_p_imp_xgb=(f1_imp_xgb/f1_baseline_xgb*100) if f1_baseline_xgb!=0 else float('inf') if f1_imp_xgb > 0 else 0.0
        acc_p_imp_xgb=(acc_imp_xgb/acc_baseline_xgb*100) if acc_baseline_xgb!=0 else float('inf') if acc_imp_xgb > 0 else 0.0
        print(f"\nXGBoost (Aug vs Base): AUC 향상: {auc_imp_xgb:+.4f} ({auc_p_imp_xgb:+.2f}%), F1 향상: {f1_imp_xgb:+.4f} ({f1_p_imp_xgb:+.2f}%), Accuracy 향상: {acc_imp_xgb:+.4f} ({acc_p_imp_xgb:+.2f}%)")

        auc_aug_ann=results_df.loc['Augmented ANN','AUC']; f1_aug_ann=results_df.loc['Augmented ANN','F1']; acc_aug_ann=results_df.loc['Augmented ANN','Accuracy']
        auc_imp_ann=(auc_aug_ann-auc_base_ann); f1_imp_ann=(f1_aug_ann-f1_base_ann); acc_imp_ann=(acc_aug_ann-acc_base_ann)
        auc_p_imp_ann=(auc_imp_ann/auc_base_ann*100) if auc_base_ann!=0 else float('inf') if auc_imp_ann > 0 else 0.0
        f1_p_imp_ann=(f1_imp_ann/f1_base_ann*100) if f1_base_ann!=0 else float('inf') if f1_imp_ann > 0 else 0.0
        acc_p_imp_ann=(acc_imp_ann/acc_base_ann*100) if acc_base_ann!=0 else float('inf') if acc_imp_ann > 0 else 0.0
        print(f"ANN (Aug vs Base): AUC 향상: {auc_imp_ann:+.4f} ({auc_p_imp_ann:+.2f}%), F1 향상: {f1_imp_ann:+.4f} ({f1_p_imp_ann:+.2f}%), Accuracy 향상: {acc_imp_ann:+.4f} ({acc_p_imp_ann:+.2f}%)")

        auc_smote_xgb = results_df.loc['SMOTE + XGBoost', 'AUC']; f1_smote_xgb = results_df.loc['SMOTE + XGBoost', 'F1']; acc_smote_xgb = results_df.loc['SMOTE + XGBoost', 'Accuracy']
        auc_imp_smote_xgb = (auc_smote_xgb - auc_baseline_xgb); f1_imp_smote_xgb = (f1_smote_xgb - f1_baseline_xgb); acc_imp_smote_xgb = (acc_smote_xgb - acc_baseline_xgb)
        auc_p_imp_smote_xgb = (auc_imp_smote_xgb/auc_baseline_xgb*100) if auc_baseline_xgb!=0 else float('inf') if auc_imp_smote_xgb > 0 else 0.0
        f1_p_imp_smote_xgb = (f1_imp_smote_xgb/f1_baseline_xgb*100) if f1_baseline_xgb!=0 else float('inf') if f1_imp_smote_xgb > 0 else 0.0
        acc_p_imp_smote_xgb = (acc_imp_smote_xgb/acc_baseline_xgb*100) if acc_baseline_xgb!=0 else float('inf') if acc_imp_smote_xgb > 0 else 0.0
        print(f"XGBoost (SMOTE vs Base): AUC 향상: {auc_imp_smote_xgb:+.4f} ({auc_p_imp_smote_xgb:+.2f}%), F1 향상: {f1_imp_smote_xgb:+.4f} ({f1_p_imp_smote_xgb:+.2f}%), Accuracy 향상: {acc_imp_smote_xgb:+.4f} ({acc_p_imp_smote_xgb:+.2f}%)")

        auc_smote_ann = results_df.loc['SMOTE + ANN', 'AUC']; f1_smote_ann = results_df.loc['SMOTE + ANN', 'F1']; acc_smote_ann = results_df.loc['SMOTE + ANN', 'Accuracy']
        auc_imp_smote_ann = (auc_smote_ann - auc_base_ann); f1_imp_smote_ann = (f1_smote_ann - f1_base_ann); acc_imp_smote_ann = (acc_smote_ann - acc_base_ann)
        auc_p_imp_smote_ann = (auc_imp_smote_ann/auc_base_ann*100) if auc_base_ann!=0 else float('inf') if auc_imp_smote_ann > 0 else 0.0
        f1_p_imp_smote_ann = (f1_imp_smote_ann/f1_base_ann*100) if f1_base_ann!=0 else float('inf') if f1_imp_smote_ann > 0 else 0.0
        acc_p_imp_smote_ann = (acc_imp_smote_ann/acc_base_ann*100) if acc_base_ann!=0 else float('inf') if acc_imp_smote_ann > 0 else 0.0
        print(f"ANN (SMOTE vs Base): AUC 향상: {auc_imp_smote_ann:+.4f} ({auc_p_imp_smote_ann:+.2f}%), F1 향상: {f1_imp_smote_ann:+.4f} ({f1_p_imp_smote_ann:+.2f}%), Accuracy 향상: {acc_imp_smote_ann:+.4f} ({acc_p_imp_smote_ann:+.2f}%)")

        # Aug vs SMOTE
        auc_change_llm_vs_smote_xgb = auc_augmented_xgb - auc_smote_xgb; f1_change_llm_vs_smote_xgb = f1_augmented_xgb - f1_smote_xgb; acc_change_llm_vs_smote_xgb = acc_augmented_xgb - acc_smote_xgb
        auc_p_change_llm_vs_smote_xgb = (auc_change_llm_vs_smote_xgb / auc_smote_xgb * 100) if auc_smote_xgb != 0 else (float('inf') if auc_change_llm_vs_smote_xgb > 0 else 0.0)
        f1_p_change_llm_vs_smote_xgb = (f1_change_llm_vs_smote_xgb / f1_smote_xgb * 100) if f1_smote_xgb != 0 else (float('inf') if f1_change_llm_vs_smote_xgb > 0 else 0.0)
        acc_p_change_llm_vs_smote_xgb = (acc_change_llm_vs_smote_xgb / acc_smote_xgb * 100) if acc_smote_xgb != 0 else (float('inf') if acc_change_llm_vs_smote_xgb > 0 else 0.0)
        print(f"XGBoost (Aug vs SMOTE): AUC 향상: {auc_change_llm_vs_smote_xgb:+.4f} ({auc_p_change_llm_vs_smote_xgb:+.2f}%), F1 향상: {f1_change_llm_vs_smote_xgb:+.4f} ({f1_p_change_llm_vs_smote_xgb:+.2f}%), Accuracy 향상: {acc_change_llm_vs_smote_xgb:+.4f} ({acc_p_change_llm_vs_smote_xgb:+.2f}%)")

        auc_change_llm_vs_smote_ann = auc_aug_ann - auc_smote_ann; f1_change_llm_vs_smote_ann = f1_aug_ann - f1_smote_ann; acc_change_llm_vs_smote_ann = acc_aug_ann - acc_smote_ann
        auc_p_change_llm_vs_smote_ann = (auc_change_llm_vs_smote_ann / auc_smote_ann * 100) if auc_smote_ann != 0 else (float('inf') if auc_change_llm_vs_smote_ann > 0 else 0.0)
        f1_p_change_llm_vs_smote_ann = (f1_change_llm_vs_smote_ann / f1_smote_ann * 100) if f1_smote_ann != 0 else (float('inf') if f1_change_llm_vs_smote_ann > 0 else 0.0)
        acc_p_change_llm_vs_smote_ann = (acc_change_llm_vs_smote_ann / acc_smote_ann * 100) if acc_smote_ann != 0 else (float('inf') if acc_change_llm_vs_smote_ann > 0 else 0.0)
        print(f"ANN (Aug vs SMOTE): AUC 향상 {auc_change_llm_vs_smote_ann:+.4f} ({auc_p_change_llm_vs_smote_ann:+.2f}%), F1 향상: {f1_change_llm_vs_smote_ann:+.4f} ({f1_p_change_llm_vs_smote_ann:+.2f}%), Accuracy 향상: {acc_change_llm_vs_smote_ann:+.4f} ({acc_p_change_llm_vs_smote_ann:+.2f}%)")

    else:
        print("결과 데이터프레임이 비어있거나 유효한 AUC/F1 점수가 없어 시각화 및 상세 향상률 출력을 건너뜁니다.")
        results_df_melted = results_df.reset_index().melt(id_vars="Model", var_name="Metric", value_name="Score")
    if is_umap:
        umap_plot(
            X_train_p=X_train_p,
            y_train_main=y_train_main,
            X_impossible_p_list=X_impossible_p_list,
            y_impossible_opposite_label_list=y_impossible_opposite_label_list,
            feature_columns=['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'] # 컬럼명 명시적 전달
        )

##main

In [ ]:
main(is_umap = False)

데이터 전처리 시작
수치형 컬럼: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
범주형 컬럼: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
데이터 전처리 완료
최종 학습 데이터 (전처리됨): (20838, 108), 레이블: 20838
최종 검증 데이터 (전처리됨): (5210, 108), 레이블: 5210
최종 테스트 데이터 (전처리됨): (6513, 108), 레이블: 6513
결정 경계 근처 (확률 0.5±0.2) 앵커 선택 중...
총 20838개 샘플 중 4157개의 결정 경계 근처 앵커 선택됨.
불가능 샘플 생성을 위한 최종 앵커 수: 4000
캐시 파일 로드 중: /content/drive/MyDrive/nlp/impossible_samples_cache_3.jsonl
5300개의 불가능 샘플을 캐시에서 로드함.
모든 필수 불가능 샘플이 캐시에 존재하거나 처리할 앵커가 없으므로 RAG 인덱스 초기화를 건너뜁니다.

4000개의 선택된 앵커에 대해 불가능 샘플 생성 시작 (캐싱 및 병렬 처리)...


앵커용 LLM 요청 준비:   0%|          | 0/4000 [00:00<?, ?it/s]

LLM 호출이 필요한 새로운 불가능 샘플 없음 (모두 캐시되었거나 처리할 앵커 없음).

총 4000개의 앵커에 대해 증강 데이터셋 구성 시도 (필터링 포함)...


증강 데이터용 불가능 샘플 처리 및 필터링:   0%|          | 0/4000 [00:00<?, ?it/s]

필터링 후, 총 1701개의 유효한 불가능 샘플을 증강에 사용합니다.
  - "고소득층(>50K)으로 레이블된 불가능 샘플" (원래 앵커는 저소득층): 1701 건
  - "저소득층(<=50K)으로 레이블된 불가능 샘플" (원래 앵커는 고소득층): 0 건
총 채택된 불가능 샘플 수: 1701 건
증강된 학습 데이터 크기: (22539, 108), 증강된 레이블 크기: 22539

[베이스라인 1] XGBoost (원본 데이터, F1 기준 조기 종료, 네이티브 API) 학습 중...
XGBoost 네이티브 API로 학습 시작...
--- Baseline 모델 (XGBoost 네이티브) 평가 결과 ---
Accuracy: 0.8746, AUC: 0.9318, F1: 0.7180, Precision: 0.7825, Recall: 0.6633

[베이스라인 2] ANN (원본 데이터) 학습 중...
Baseline ANN (Original Data) 학습 시작 (조기 종료 기능 포함, 샘플 가중치 적용 가능, F1 스코어 기준 최적화)...


...

ANN 에폭 18 [검증]:   0%|          | 0/82 [00:00<?, ?it/s]

ANN 에폭 18 - 학습 손실 (가중 적용): 0.2868, 검증 손실: 0.3226, 검증 F1: 0.6897, (참고용 검증 AUC: 0.9065)
ANN 검증 F1 스코어 개선되지 않음. (9/10)


ANN 에폭 19 [학습]:   0%|          | 0/352 [00:00<?, ?it/s]

ANN 에폭 19 [검증]:   0%|          | 0/82 [00:00<?, ?it/s]

ANN 에폭 19 - 학습 손실 (가중 적용): 0.2873, 검증 손실: 0.3219, 검증 F1: 0.6822, (참고용 검증 AUC: 0.9064)
ANN 검증 F1 스코어 개선되지 않음. (10/10)
10 에폭 동안 ANN 검증 F1 스코어 개선 없어 조기 종료합니다.
Augmented ANN 학습 완료.
가장 좋았던 검증 F1 (0.6905) 시점의 모델로 복원합니다.
--- Augmented ANN 평가 결과 ---
Accuracy: 0.8601, AUC: 0.9164, F1: 0.6982, Precision: 0.7264, Recall: 0.6722

--- 최종 성능 비교 (테스트 데이터 기준) ---
               Model       AUC        F1  Accuracy
0   Baseline XGBoost  0.931821  0.717984  0.874559
1       Baseline ANN  0.916296  0.696667  0.860279
2    SMOTE + XGBoost  0.929540  0.730366  0.873484
3        SMOTE + ANN  0.915409  0.711084  0.855520
4  Augmented XGBoost  0.931694  0.722165  0.875480
5      Augmented ANN  0.916377  0.698244  0.860126

XGBoost (Aug vs Base): AUC 향상: -0.0001 (-0.01%), F1 향상: +0.0042 (+0.58%), Accuracy 향상: +0.0009 (+0.11%)
ANN (Aug vs Base): AUC 향상: +0.0001 (+0.01%), F1 향상: +0.0016 (+0.23%), Accuracy 향상: -0.0002 (-0.02%)
XGBoost (SMOTE vs Base): AUC 향상: -0.0023 (-0.24%), F1 향상: +0.0124 (+1.72%), Accuracy 향상: